# How to attach variables to step output

* **Difficulty level**: intermediate
* **Time need to lean**: 15 minutes or less
* **Key points**:
  * 
  

## Reminder

If a SoS step contains multiple substeps, defined by options `group_by` or `for_each`, the `_input` becomes the groups of `step_input` and `_output` becomes the groups of `step_output`, and the steps are executed for each of the groups.

Moreover, the group information of `step_output` will be passed as the default input to the next step in a simple forward-style workflow, or as input to another step with functions `output_from` or `named_output`. As shown in the following example, the `step_output` of step `A` becomes the input of step `B`, creating two substeps.

In [3]:
!touch a.txt b.txt

%run B -v0

[A]
input: 'a.txt', 'b.txt', group_by=1
output: _input.with_suffix('.bak')

_output.touch()
print(f'step_input={step_input}, _input={_input}, _output={_output}')

[B]
input: output_from('A')
print(f'step_output={step_input}, _output={_input}')


,B,Workflow ID 62023b76ae137fa8,Index #3,completed Ran for < 5 seconds


step_input=a.txt b.txt, _input=b.txt, _output=b.bak
step_input=a.txt b.txt, _input=a.txt, _output=a.bak
step_output=a.bak b.bak, _output=a.bak
step_output=a.bak b.bak, _output=b.bak


## Attaching group information to `step_output`

As we

In [ ]:

%run B -v0

[A]
input: 'a.txt', 'b.txt', group_by=1
output: _input.with_suffix('.bak')

_output.touch()
print(f'step_input={step_input}, _input={_input}, _output={_output}')

[B]
input: output_from('A')
print(f'step_output={step_input}, _output={_input}')


<div class="bs-callout bs-callout-primary" role="alert">
    <h4>The <code>group_with</code> output option</h4>
  <p></p>  
</div>

## The `group_with` output option

Just like the `group_with` option of the `input` statement, the `group_with` output option assigns a sequence of variables to each of the output groups (variable `_output`). 

In [14]:
!touch a.txt b.txt

%run -v0

samples = ['A', 'B']

[1]
input: 'a.txt', 'b.txt', group_by=1
output: _input.with_suffix('.bak'), group_with=dict(sample=samples)

_output.touch()
print(f'step_input={step_input}, _input={_input}, _output={_output}, _output.sample={_output.sample}')

[2]
print(f'step_output={step_input}, _output={_input}, _output.sample={_input.sample}')


,default,Workflow ID 16bb70a4e801bcd0,Index #13,completed Ran for 0 sec


step_input=a.txt b.txt, _input=a.txt, _output=a.bak, _output.sample=A
step_input=a.txt b.txt, _input=b.txt, _output=b.bak, _output.sample=B
step_output=a.bak b.bak, _output=a.bak, _output.sample=A
step_output=a.bak b.bak, _output=b.bak, _output.sample=B


## Further reading

* 